In [243]:
import pandas as pd

In [244]:
bridges = pd.read_excel('../data/raw/BMMS_overview.xlsx')    

In [245]:
#only N1 bridges
bridges_n1 = bridges[bridges['road'] == 'N1']

In [246]:
#filter out bridges with same LRPName keeping the one with worst condition
bridges_n1 = bridges_n1.sort_values('condition', ascending=False).drop_duplicates('LRPName')

In [247]:
selected_columns = ['road', 'type', 'length', 'condition', 'LRPName','name','chainage','lat','lon']
bridges_n1 = bridges_n1[selected_columns]
bridges_n1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 639 entries, 19382 to 393
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   road       639 non-null    object 
 1   type       639 non-null    object 
 2   length     639 non-null    float64
 3   condition  639 non-null    object 
 4   LRPName    639 non-null    object 
 5   name       639 non-null    object 
 6   chainage   639 non-null    float64
 7   lat        639 non-null    float64
 8   lon        639 non-null    float64
dtypes: float64(4), object(5)
memory usage: 49.9+ KB


In [248]:
bridges_n1['type'] = 'bridge'
#sort bridges on chainage
bridges_n1 = bridges_n1.sort_values('chainage')

In [249]:
#filter all bridges with chainage higher than 232.754 - start of Chittagong
bridges_n1 = bridges_n1[bridges_n1['chainage'] < 232.754]
#size of bridges_n1
bridges_n1.shape


(221, 9)

In [250]:
#if chainage of bridges is the same: put them together with average length
bridges_n1 = bridges_n1.groupby('chainage').agg({'road':'first', 'type':'first', 'length':'mean', 'condition':'first', 'LRPName':'first', 'name':'first','lat':'first','lon':'first'}).reset_index()

In [251]:
#cut last character of LRPName
bridges_n1['LRPName'] = bridges_n1['LRPName'].str[:-1]
#if LRPName is the same: put them together with average length and worst condition
bridges_n1 = bridges_n1.groupby('LRPName').agg({'road':'first', 'type':'first', 'length':'mean','chainage':'first', 'condition':'max', 'name':'first','lat':'first','lon':'first'}).reset_index()



In [252]:
bridges_n1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 151 entries, 0 to 150
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LRPName    151 non-null    object 
 1   road       151 non-null    object 
 2   type       151 non-null    object 
 3   length     151 non-null    float64
 4   chainage   151 non-null    float64
 5   condition  151 non-null    object 
 6   name       151 non-null    object 
 7   lat        151 non-null    float64
 8   lon        151 non-null    float64
dtypes: float64(4), object(5)
memory usage: 10.7+ KB


In [253]:
bridge_dict = bridges_n1.to_dict('records')

In [254]:
for i,bridge in enumerate(bridge_dict):
    #add source at start of road
    if i == 0:
        bridge_dict.insert(0, {'road':'N1', 'type':'source', 'length':0, 'condition':0, 'LRPName':bridge['LRPName'],'name':'road N1','chainage':0,'lat':bridge['lat'],'lon':bridge['lon']})
        bridge_dict.insert(1, {'road':'N1', 'type':'link', 'length':bridge_dict[i+1]['chainage']*1000, 'condition':0, 'LRPName':bridge['LRPName']+'-road','name':'road N1','chainage':0,'lat':bridge['lat'],'lon':bridge['lon']})
    #add links in between bridges
    elif i%2 == 0 and i < len(bridge_dict)-1:
        #add link between bridges
        road_length = (bridge_dict[i+1]['chainage']-bridge['chainage'])*1000-bridge['length'] if (bridge_dict[i+1]['chainage']-bridge['chainage'])*1000-bridge['length'] > 0 else 0
        bridge_dict.insert(i+1, {'road':'N1', 'type':'link', 'length':road_length, 'condition':0, 'LRPName':bridge['LRPName']+'-road','name':'road N1','chainage':bridge['chainage']+bridge['length']/1000,'lat':bridge['lat'],'lon':bridge['lon']})
    #add sink at end of road
    elif i == len(bridge_dict)-1 and bridge['type']!='sink':        
        bridge_dict.append({'road':'N1', 'type':'link', 'length':(232.754-bridge['chainage'])*1000-bridge['length'], 'condition':0,'LRPName':bridge['LRPName']+'-road','name':'road N1','chainage':bridge['chainage']+bridge['length']/1000,'lat':bridge['lat'],'lon':bridge['lon']}) 
        bridge_dict.append({'road':'N1', 'type':'sink', 'length':0, 'condition':0, 'LRPName':'LRP235','name':'road N1','chainage':232.754,'lat':bridge['lat'],'lon':bridge['lon']}) 
    

In [255]:
#dict to dataframe
bridges_n1_cleaned = pd.DataFrame(bridge_dict)

In [256]:
bridges_n1_cleaned.tail()

,road,type,length,condition,LRPName,name,chainage,lat,lon
300,N1,bridge,8.3,A,LRP233,GLAXO BOX CULVERT,230.9210,22.379701,91.767048
301,N1,link,1036.7,0,LRP233-road,road N1,230.9293,22.379701,91.767048
302,N1,bridge,3.0,A,LRP234,CORNAIL HAT BOX CULVERT,231.9660,22.369684,91.773926
303,N1,link,785.0,0,LRP234-road,road N1,231.9690,22.369684,91.773926
304,N1,sink,0.0,0,LRP235,road N1,232.7540,22.369684,91.773926


In [260]:
#road,id,model_type,name,lat,lon,length,condition
#rename type to model_type
bridges_n1_cleaned = bridges_n1_cleaned.rename(columns={'type': 'model_type'})
#insert column id with index
bridges_n1_cleaned.insert(1, 'id', bridges_n1_cleaned.index)

In [261]:
#save to csv
bridges_n1_cleaned.to_csv('../model/input/infrastructure.csv', index=False)